# 4.2 Implementation - Variational Inference

Latent Dirichlet Allocation was also implemented using variational inference. In situations where variational inference is typically used, the posterior is typically intractable to calculate directly. In the case of LDA, the posterior $p(\theta,z,w | \alpha,\beta)$ is difficult to compute, so the distribution is instead approximated with the variational distribution:

$$q(\theta,z | \gamma,\phi) = q(\theta|\gamma) \prod_{n=1}^{N} q(z_n|\phi_n)$$

Using Jensen's inequality, it can be shown that the difference between the log likelihood of the true posterior and the variational approximation is the KL-divergence between the two. In order words:

$$ \log(p(w|\alpha,\beta) = L(\gamma,\phi;\alpha,\beta) + D(q(\theta,z|\gamma,\phi)||p(\theta,z|w,\alpha,\beta))$$

We can choose to either minimize the KL-divergence or maximize the likelihood. Here, the latter is approach is taken. Factoring the likelihood appropriately, we can write the following:

$$ L(\gamma,\phi;\alpha,\beta) = E_q[\log p(\theta|\alpha)] + E_q[\log p(z|\theta)] + E_q [\log p(w|z,\beta)] - E_q [\log q(\theta)] - E_q[\log q(z)] $$

This likelihood is maximized through Expectation-Maximization (EM). During the expectation step, the variational parameters $\phi$ and $\gamma$ are first optimized by maximizing the likelihood with respect to each individually. During the maximization step, the likelihood is then maximized with respect to model parameters $\alpha$ and $\beta$. This process is outlined below.

### Variables and Parameters

document:    $m = 1,...,M$

topic:       $z = 1,...,k$

word:        $w = 1,...,N_m$

vocabulary : $v = 1,...,V$

$\alpha: 1 \times k$ Model parameter - vector of topic distribution probabilities for each document

$\beta: k \times v$ Model parameter - matrix of word probabilities for each topic

$\phi: M \times N_m \times k$ Variational parameter - matrix of topic probabilities for each word in each document

$\gamma: M \times k$ Variational parameter - matrix of topic probabilities for each document

#### Import packages and functions

In [1]:
import numpy as np
from numpy import sqrt,mean,square
from scipy.special import digamma, polygamma

### Optimize variational parameters $\phi$ and $\gamma$

By taking the derivative the log likelihood with respect to $\phi$ and setting the result to zero, we find the maximal value of $\phi$:

$$ \phi_{ni} \propto \beta_{iv} \exp(\Psi(\gamma_i) - \Psi(\sum_{j=1}^k(\gamma_j)) $$

where $\beta_{iv}$ = $p(w_n^v = 1|z_n = i)$ and $\psi$ is the digamma function (derivative of the log gamma function $\Gamma$). As $\phi$ represents the probability of each word in a document for each state, these values must be normalized such that each row representing a word position within a document must sum to 1.


In a similar fashion, it can be shown that $\gamma$ is maximized at:

$$ \gamma_i = \alpha_i + \sum_{n=1}^N(\phi_{ni})$$

In [2]:
## Optimize variational parameter phi
def opt_phi(beta,gamma,words,M,N,k):
    for m in range(M):
        for n in range(N[m]):
            for i in range(k):
                phi[m][n,i] = beta[words[m][n],i] * np.exp(digamma(gamma[m,i]) - digamma(np.sum(gamma[m,:])))
            # Normalize across states so phi represents probability over states for each word
            phi[m][n,:] = phi[m][n,:]/np.sum(phi[m][n,:])
    return phi


## Optimize variational parameter gamma
def opt_gamma(alpha,phi,M):
    gamma = np.tile(alpha,(M,1)) + np.array(list(map(lambda x: np.sum(x,axis=0),phi)))
    return gamma

### Estimate model parameters $\alpha$ and $\beta$

By taking the derivative of the log likelihood and applying the appropriate Lagrange multipliers to ensure probabilities sum to 1, we find that $/beta$ is maximized with:

$$ \beta_{ij} \propto \sum_{m=1}^M \sum_{n=1}^{N_m} \phi_{dni}w_{mn}^j$$

where $w_{mn}^j$ = 1 if the $n^{th}$ word of document $m$ is equal to $j$, and 0 otherwise. Since the columns of \beta represent the probability of each word given the topic of that particular column, they must be normalized to sum to 1.

Taking the derivative of the log likelihood with respect to $\alpha$ yields:

$$ \frac{\partial L}{\partial\alpha_i} = M(\Psi(\sum_{j=1}^k\alpha_j)-\Psi(\alpha_i)) - \sum_{m=1}^M(\Psi(\gamma_{di})-\Psi(\sum_{j=1}^k\gamma_{dj}))$$

Because this is difficult to find the zero intercept of this derivative, $\alpha$ is instead maximized numerically with the Newton-Raphson method. The Hessian is of the form:

$$ \frac{\partial^2 L}{\partial\alpha_i\partial\alpha_j} = M(\Psi'(\sum_{j=1}^k \alpha_j) - \delta(i,j)\Psi'(\alpha_i))$$ 

Note: This is slightly different from what is stated in the paper, which has a couple errors in the reported form of the Hessian.

In [3]:
## Optimize beta
def est_beta(phi,words,k,V):
    for j in range (V):
        # Construct w_mn == j of same shape as phi
        w_mnj = [np.tile((word == j),(k,1)).T for word in words]
        beta[j,:] = np.sum(np.array(list(map(lambda x: np.sum(x,axis=0),phi*w_mnj))),axis=0)
        
    # Normalize across states so beta represents probability of each word given the state
    for i in range(k):
        beta[:,i] = beta[:,i]/sum(beta[:,i])
        
    return beta


## Optimize alpha
#  (Newton-Raphson method, for a Hessian with special structure)
def est_alpha(alpha,gamma,M,k,nr_max_iters = 1000,tol = 10**-2.0):
    for it in range(nr_max_iters):
        alpha_old = alpha
        
        #  Calculate gradient 
        g = M*(digamma(np.sum(alpha))-digamma(alpha)) + np.sum(digamma(gamma)-np.tile(digamma(np.sum(gamma,axis=1)),(k,1)).T,axis=0)
        #  Calculate Hessian diagonal component
        h = -M*polygamma(1,alpha) 
        #  Calculate Hessian constant component
        z = M*polygamma(1,np.sum(alpha))
        #  Calculate constant
        c = np.sum(g/h)/(z**(-1.0)+np.sum(h**(-1.0)))

        #  Update alpha
        alpha = alpha - (g-c)/h
        
        #  Check convergence
        if sqrt(mean(square(alpha-alpha_old)))<tol:
            break
        
    return alpha

# 5.2 Tests - Variational Inference 

### Set data set characteristics
To test if our implementation of latent dirichlet allocation with variational inference works, we first generate some toy data. This toy data set will consist of 300 documents, each with a uniform random length between 150 and 200 words. The size of the vocabulary of words in the documents is set to be 30, assumed to be generated from 10 topics.

In [4]:
np.random.seed(1337)

M = 300
k = 10
N = np.random.randint(150,200,size=M)
V = 30

print('N: {0}'.format(N))

N: [173 178 190 189 175 189 176 168 170 158 159 156 176 173 174 151 177 179
 156 172 152 191 190 161 151 173 169 196 167 197 177 153 170 158 158 157
 177 159 154 181 183 162 156 154 196 170 168 176 153 191 184 192 154 158
 164 188 153 174 179 158 197 157 154 185 168 159 199 178 163 162 198 195
 183 195 150 199 159 170 195 184 198 198 177 157 170 171 188 194 150 166
 168 155 191 175 198 179 173 169 156 160 195 160 195 166 177 177 153 191
 162 195 165 150 162 157 161 151 188 183 190 178 159 154 157 183 157 181
 160 157 172 153 161 155 192 165 180 191 170 167 150 173 173 152 154 154
 191 156 199 188 181 179 162 164 173 159 178 150 187 167 168 177 155 184
 167 196 193 167 151 169 157 154 157 194 172 194 156 191 194 180 186 186
 152 197 156 151 163 180 166 174 166 158 179 169 176 195 177 188 151 169
 153 187 191 184 189 181 194 172 171 188 151 164 188 180 151 177 187 197
 150 164 167 152 182 186 163 191 155 151 183 197 173 165 187 154 154 172
 181 198 194 181 180 192 193 155 159 184 151 193

### Generate data
The documents are then generated one by one according to the LDA model (see 2 Algorithm description). Three distinct groups of documents are generated: the first 100 have a strong preference for topics 1, 2, and 3; the second 100 have a strong preference for topics 4, 5, and 6; and the last 100 have a strong preference for topics 7, 8, 9, and 10. Furthermore, each topic will have a strong preference for 3 words, such that each word is prevalent in one topic. The structure of the resulting parameters are shown in Figures 1 and 2 below.

In [5]:
# Create 3 groups of documents, each with a topic preference
alpha_gen1 = np.array((20,15,10,1,1,1,1,1,1,1))
alpha_gen2 = np.array((1,1,1,10,15,20,1,1,1,1))
alpha_gen3 = np.array((1,1,1,1,1,1,10,12,15,18))

# Arbitrarily choose each topic to have 3 very common words
beta_probs = np.ones((V,k)) + np.array([np.arange(V)%k==i for i in range(k)]).T*19
beta_gen = np.array(list(map(lambda x: np.random.dirichlet(x),beta_probs.T))).T

w_struct = list();
theta = np.empty((M,k))

# Generate each document
for m in range(M):
    # Draw topic distribution for the document
    if m<M/3:
        theta[m,:] = np.random.dirichlet(alpha_gen1,1)[0]
    elif m<2*M/3:
        theta[m,:] = np.random.dirichlet(alpha_gen2,1)[0]
    else:
        theta[m,:] = np.random.dirichlet(alpha_gen3,1)[0]
    doc = np.array([])
    
    for n in range(N[m]):
        # Draw topic according to document's topic distribution
        z_n = np.random.choice(np.arange(k),p=theta[m,:])
        # Draw word according to topic
        w_n = np.random.choice(np.arange(V),p=beta_gen[:,z_n])
        doc = np.append(doc,w_n)
    w_struct.append(doc)

![beta_gen](https://github.com/gytcrt/FinalProject663/blob/master/Output_Data/beta_gen.png?raw=true)

<center>Figure 1: The matrix $\beta_{gen}$ used to set the topic-word probabilities of the test data. Note how each topic strongly prefers 3 words.</center>

![theta_gen](https://github.com/gytcrt/FinalProject663/blob/master/Output_Data/theta.png?raw=true)

<center>Figure 2: The matrix $\theta$ generated by the model. Note how there are 3 distinct groups, each with their preferred topics.</center>

### Initialize parameters $\alpha, \beta, \phi$ and $\gamma$
The model and variational parameters are then randomly initialized to reasonable values:

In [6]:
alpha = 100*np.random.dirichlet(10*np.ones(k),1)[0]
beta = np.random.dirichlet(np.ones(V),k).T

phi = np.array([1/k*np.ones([N[m],k]) for m in range(M)])
gamma = np.tile(alpha,(M,1)) + np.tile(N/k,(k,1)).T

### Expectation Maximization (EM)

#### Convergence Criterion
The variational inference parameter $\gamma$ contains the topic likelihoods of every document. As such, $\gamma$ identifies to which group a document is likely to belong. As such, the convergence criterion was chosen to monitor this parameter. The root-mean-square of the change in $\gamma$ is calculated on every iteration of EM and compared against a tolerance parameter.

In [7]:
def converged(gamma,gamma_old,convergence):
    print(sqrt(mean(square(gamma-gamma_old))))
    return sqrt(mean(square(gamma-gamma_old))) < convergence

#### Inference by iterative EM

Expectation-Maximization is carried out by consecutively maximizing each of the four parameters $\alpha, \beta, \phi$ and $\gamma$ with respect to the log likelihood until either the convergence criterion has been met or a maximimum number of iterations have been calculated.

In [9]:
convergence = 5*10**(-2.0)
successfully_Converged = False
max_iters = 10**3

for iters in range(max_iters):
    #print(iters)
    gamma_old = gamma
    
    ## Expectation step: Update variational parameters
    phi   = opt_phi(beta,gamma,w_struct,M,N,k)
    gamma = opt_gamma(alpha,phi,M)
    
    ## Maximization step: Update model parameters
    beta  = est_beta(phi,w_struct,k,V)
    alpha = est_alpha(alpha,gamma,M,k)
    
    if converged(gamma,gamma_old,convergence):
        successfully_Converged = True
        break

0.0482680757129


/Users/andrea/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


### Results
After running until the root-mean-square of the difference in $\gamma$ dropped below 0.05, the algorithm was considered converged and terminated. The results are visualized in plots below in Figures 3 and 4. Since the model parameter $\beta$ estimated by the algorithm should correspond witht the $\beta$ used to generate the data, and inferred variational parameter $\gamma$ should correspond to $\theta$, Figures 3 and 4 should resemble Figures 1 and 2 respectively.

![beta_est_unordered](https://github.com/gytcrt/FinalProject663/blob/master/Output_Data/beta_est_unordered.png?raw=true)

<center>Figure 3: The matrix $\beta_{est}$ inferred by the algorithm.</center>

![gamma_unordered](https://github.com/gytcrt/FinalProject663/blob/master/Output_Data/gamma_unordered.png?raw=true)

<center>Figure 4: The matrix $\gamma$ inferred by the algorithm.</center>

At first, Figures 1 and 2 do not appear to match up with Figures 3 and 4. However, the individual topic identities do not have any specific relation to their index. In other words, there is a non-identifiability issue at play here. It is however apparent in Figure 4 that the algorithm correctly identifies 3 separate groups of documents of the right size. By visually inspecting $\gamma$ and re-arranging the order of the topics, we get Figures 5 and 6 below:

![beta_est_rearranged](https://github.com/gytcrt/FinalProject663/blob/master/Output_Data/beta_est_rearranged.png?raw=true)

<center>Figure 5: The matrix $\beta_{est}$ inferred by the algorithm, rearranged in original topic order.</center>

![gamma_rearranged](https://github.com/gytcrt/FinalProject663/blob/master/Output_Data/gamma_rearranged.png?raw=true)

<center>Figure 6: The matrix $\gamma$ inferred by the algorithm, rearranged in original topic order.</center>

While not perfect, the visualizations of $\beta$ and $\gamma$ with the topics re-arranged now exhibit patterns similar to the original structures visible in Figures 1 and 2. In particular, the three diagonals in $\beta$ representing the three preferred words of each topic can be clearly seen in Figure 5, and the three boxes corresponding to the preferred topic distributions of the three groups of documents are also apparent in Figure 6. With more data (either more documents or more words in each document), these structures are likely to be recovered with even higher accuracy. 